In [1]:
from google.colab import files
uploaded=files.upload()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
model = pd.read_csv('train.csv')
check = pd.read_csv('predict.csv')

Saving train.csv to train.csv
Saving predict.csv to predict.csv
Saving example.csv to example.csv


In [2]:
print(model.shape)
model.info()
print(model.isna().sum())
print("__________")
print(check.shape)
check.info()
print(check.isna().sum())
print("__________")

(31641, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31641 entries, 0 to 31640
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Vmag         31641 non-null  float64
 1   Plx          31641 non-null  float64
 2   e_Plx        31641 non-null  float64
 3   B-V          31641 non-null  float64
 4   SpType       31641 non-null  object 
 5   Amag         31641 non-null  float64
 6   TargetClass  31641 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.7+ MB
Vmag           0
Plx            0
e_Plx          0
B-V            0
SpType         0
Amag           0
TargetClass    0
dtype: int64
__________
(7911, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7911 entries, 0 to 7910
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Vmag    7911 non-null   float64
 1   Plx     7911 non-null   float64
 2   e_Plx   7911 non-null   float

In [4]:
train=model.copy()
train["Температура"]=0
train["Светимость"]=0
for index, row in train.iterrows():
    train.loc[index, "Температура"] = re.findall(r'([A-Z])', train.loc[index, "SpType"])[0]
    try:
        n = re.findall(r'.*[0-9]([IV]*)', train.loc[index, "SpType"])[-1]
        if n not in ["I", "II", "III", "IV", "V", "VI", "VII"]:
            train.loc[index, "Светимость"] = "IV"
        else:
            train.loc[index, "Светимость"] = n
    except IndexError:
        train.loc[index, "Светимость"] = "IV"
train.drop("SpType", axis=1, inplace=True)

train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,Температура,Светимость
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,G,V
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,G,I
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,F,V
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,K,III
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,K,III
...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,A,II
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,G,III
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,F,IV
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,K,III


In [6]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
label_1 = le1.fit_transform(train["Температура"])
label_2 = le2.fit_transform(train["Светимость"])
train.drop("Температура", axis=1, inplace=True)
train.drop("Светимость", axis=1, inplace=True)
train["Температура"] = label_1
train["Светимость"] = label_2

train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,Температура,Светимость
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,4,4
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,4,0
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,3,4
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,5,2
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,5,2
...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,0,1
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,4,2
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,3,3
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,5,2


In [7]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
x = train.drop(columns=["TargetClass"])
y = train["TargetClass"]
x_tr, x_te, y_tr, y_te = train_test_split(x, y, random_state=42, test_size=0.3)
rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc.fit(x_tr, y_tr)

CPU times: user 4.84 s, sys: 63.8 ms, total: 4.91 s
Wall time: 3.33 s


RandomForestClassifier(n_jobs=-1, random_state=42)

In [9]:
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
pred = rfc.predict(x_te)
roc_auc_score(y_te, pred)

0.9587146004795541

In [10]:
test=check.copy()
test["Температура"] = 0
test["Светимость"] = 0
for index, row in test.iterrows():
    test.loc[index, "Температура"] = re.findall(r'([A-Z])', test.loc[index, "SpType"])[0]
    try:
        n = re.findall(r'.*[0-9]([IV]*)', test.loc[index, "SpType"])[-1]
        if n not in ["I", "II", "III", "IV", "V", "VI", "VII"]:
            test.loc[index, "Светимость"] = "IV"
        else:
            test.loc[index, "Светимость"] = n
    except IndexError:
        test.loc[index, "Светимость"] = "IV"
test.drop("SpType", axis=1, inplace=True)
test

,Vmag,Plx,e_Plx,B-V,Amag,Температура,Светимость
0,4.039519,13.596504,0.845101,-0.097823,15.774122,F,V
1,6.655719,4.010332,1.050017,1.386670,15.344976,K,III
2,6.405060,0.348855,0.961598,0.951503,12.236103,B,IV
3,9.113068,2.954461,1.251422,1.002819,15.560838,K,III
4,6.107686,3.144180,0.475795,0.493345,12.771981,G,II
...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,M,III
7907,6.255358,3.034436,0.449386,0.966053,13.898403,K,III
7908,8.643598,10.932145,0.947897,0.504383,18.980729,G,V
7909,9.400810,6.279361,1.332124,0.351073,16.872491,F,V


In [11]:
le1 = LabelEncoder()
le2 = LabelEncoder()
label_1 = le1.fit_transform(test["Температура"])
label_2 = le2.fit_transform(test["Светимость"])
test.drop("Температура", axis=1, inplace=True)
test.drop("Светимость", axis=1, inplace=True)
test["Температура"] = label_1
test["Светимость"] = label_2
test

,Vmag,Plx,e_Plx,B-V,Amag,Температура,Светимость
0,4.039519,13.596504,0.845101,-0.097823,15.774122,3,4
1,6.655719,4.010332,1.050017,1.386670,15.344976,5,2
2,6.405060,0.348855,0.961598,0.951503,12.236103,1,3
3,9.113068,2.954461,1.251422,1.002819,15.560838,5,2
4,6.107686,3.144180,0.475795,0.493345,12.771981,4,1
...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,6,2
7907,6.255358,3.034436,0.449386,0.966053,13.898403,5,2
7908,8.643598,10.932145,0.947897,0.504383,18.980729,4,4
7909,9.400810,6.279361,1.332124,0.351073,16.872491,3,4


In [12]:
pred = pd.Series(rfc.predict(test))
pred.to_csv("pred.csv")
predf = pd.read_csv("/content/pred.csv")
predf

,Unnamed: 0,0
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
7906,7906,0
7907,7907,0
7908,7908,1
7909,7909,1


In [14]:
predf.rename(columns={"Unnamed: 0": "Index", "0": "TargetClass"}, inplace=True)
test = test.join(predf)
test

,Vmag,Plx,e_Plx,B-V,Amag,Температура,Светимость,Index,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,3,4,0,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,5,2,1,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,1,3,2,1
3,9.113068,2.954461,1.251422,1.002819,15.560838,5,2,3,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,4,1,4,0
...,...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,6,2,7906,0
7907,6.255358,3.034436,0.449386,0.966053,13.898403,5,2,7907,0
7908,8.643598,10.932145,0.947897,0.504383,18.980729,4,4,7908,1
7909,9.400810,6.279361,1.332124,0.351073,16.872491,3,4,7909,1


In [15]:
final_pred=test.drop(["Index"], axis=1)
final_pred=final_pred.reindex(columns=["Vmag", "Plx", "e_Plx", "B-V", "SpType", "Amag", "TargetClass"])
final_pred.to_csv("final_pred.csv", index=False)
final_pred

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,NaN,15.774122,1
1,6.655719,4.010332,1.050017,1.386670,NaN,15.344976,0
2,6.405060,0.348855,0.961598,0.951503,NaN,12.236103,1
3,9.113068,2.954461,1.251422,1.002819,NaN,15.560838,0
4,6.107686,3.144180,0.475795,0.493345,NaN,12.771981,0
...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,NaN,16.438014,0
7907,6.255358,3.034436,0.449386,0.966053,NaN,13.898403,0
7908,8.643598,10.932145,0.947897,0.504383,NaN,18.980729,1
7909,9.400810,6.279361,1.332124,0.351073,NaN,16.872491,1
